In [1]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg') # workaround, there may be a better way
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats.distributions as di

In [2]:
url = "nhanes_2015_2016.csv"
da = pd.read_csv(url)

da["SMQ020x"] = da.SMQ020.replace({1: "Yes", 2: "No", 7: np.nan, 9: np.nan})
da["RIAGENDRx"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})


### Hypothesis Tests for One Proportion

The most basic hypothesis test may be the one-sample test for a proportion.  This test is used if we have specified a particular value as the null value for the proportion, and we wish to assess if the data are compatible with the true parameter value being equal to this specified value.  One-sample tests are not used very often in practice, because it is not very common that we have a specific fixed value to use for comparison. For illustration, imagine that the rate of lifetime smoking in another country was known to be 40%, and we wished to assess whether the rate of lifetime smoking in the US were different from 40%.  In the following notebook cell, we carry out the (two-sided) one-sample test that the population proportion of smokers is 0.4, and obtain a p-va

In [10]:
best_estimate=0.4
hypo_estimate=np.mean(da['SMQ020x'].dropna()=="Yes")
se=np.sqrt((best_estimate*(1-best_estimate))/len(da))
test_stat=(best_estimate-hypo_estimate)/se
print(test_stat)
pvalue=di.norm.cdf(test_stat)+1-di.norm.cdf(-test_stat)
print(pvalue)
print("Fail to Reject Null Hypo")

-0.7830393680037091
0.433603994264955
Fail to Reject Null Hypo


In [12]:
test_stat,pvalue=sm.stats.ztest(da['SMQ020x'].dropna()=="Yes",value=0.4)
print(test_stat,pvalue)
if pvalue>0.05:
    print("Fail to Reject Null Hypothesis")
else:
    print("Reject Null Hypothesis")

0.7806837045732543 0.43498854710143553
Fail to Reject Null Hypothesis


### Hypothesis Tests for Two Proportions

Comparative tests tend to be used much more frequently than tests comparing one population to a fixed value.  A two-sample test of proportions is used to assess whether the proportion of individuals with some trait differs between two sub-populations.  For example, we can compare the smoking rates between females and males. Since smoking rates vary strongly with age, we do this in the subpopulation of people between 20 and 25 years of age.  In the cell below, we carry out this test without using any libraries, implementing all the test procedures covered elsewhere in the course using Python code.  We find that the smoking rate for men is around 10 percentage points greater than the smoking rate for females, and this difference is statistically significant (the p-value is around 0.01).

In [17]:

results=da.groupby("RIAGENDRx").agg({"SMQ020x":[lambda x:np.mean(x=="Yes"),np.size]})
results.columns=['Proportion','Size']
print(results)

           Proportion  Size
RIAGENDRx                  
Female       0.304435  2976
Male         0.512142  2759


In [22]:
# Male-Female
print("Manual Calculation")
p_comb=(results.Proportion.Male+results.Proportion.Female)/2
hypo_estimate=results.Proportion.Male-results.Proportion.Female
best_estimate=0
val=p_comb*(1-p_comb)
se=np.sqrt(val*((1/results.Size.Female)+(1/results.Size.Male)))
t_stat=(best_estimate-hypo_estimate)/se
pvalue=di.norm.cdf(t_stat)+1-di.norm.cdf(-t_stat)
print(t_stat,pvalue)

-15.989587754738997 0.0


In [31]:
print("Using StatModels")
male=dx.loc[dx['RIAGENDRx']=='Male','SMQ020x'].replace({"Yes":1,"No":0})
female=dx.loc[dx['RIAGENDRx']=='Female','SMQ020x'].replace({"Yes":1,"No":0})
test_stat,pvalue=sm.stats.ztest(male,female)
print(test_stat,pvalue)

16.420585558984445 1.362455350780943e-60
